# Topic modelling

The goal is to find three topics from the collection, the firt one regarding drugs, the second weapons, and the third investigation. 

We start with a classic model to then test the guided lda approach. We do not expect the first one to find the three topics we want, while the second should guide the topic modelling towards the required goal.

References:
https://medium.com/analytics-vidhya/how-i-tackled-a-real-world-problem-with-guidedlda-55ee803a6f0d


In [9]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("..")

import json
import numpy as np

# from gensim.models import LsiModel, LdaMulticore
# from gensim.test.utils import common_dictionary, common_corpus
from sklearn.decomposition import LatentDirichletAllocation as lda


from lda import guidedlda as glda

# import guidedlda

# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()

from src.dataset import Dataset
from src.vectorizers import TokenVectorizer

In [15]:
dataset = Dataset()
# load only the year specified
tokens = dataset.load_dataset(year=1820, tokens=True)
# load all years
# tokens = dataset.load_dataset(tokens=True)

### Filtra i dati, se vuoi

In [16]:
tokens = [[w for w in doc if len(w) >= 3] for doc in tokens]

### Vectorize the documents
The vectorized is a tfidf one, we use the output to fit the lda model.

In [17]:
dv = TokenVectorizer(tokens, method="count")

vectors = dv.vectors()
dv.save_vectors_vectorizer(vectors)
print(f"Vocabulary length: {len(dv.vectorizer.vocabulary_)}")

Vocabulary length: 7462


In [18]:
vectors, vectorizer = TokenVectorizer.load_vectors_vectorizer(method="count")

## Classic LDA model

The number of topics is set to three, while alpha and beta have values proposed in the literature. 

Griffiths TL, Steyvers M (2004). “Finding Scientific Topics.” Proceedings of the National Academy of Sciences of the United States of America, 101, 5228–5235.

In [19]:
numTopics = 4
# alpha = 50/numTopics
alpha = 0.1
beta = 0.01

lda_model = lda(n_components = numTopics, 
                doc_topic_prior= alpha, 
                topic_word_prior = beta, 
                random_state=0, 
                n_jobs=-1)

lda_output = lda_model.fit_transform(vectors)

### Topics relevant words

The next step is to check the words for each topic, results are interesting and expected, bu twe can't see a distinction between the topics we want.

In [20]:
n_top_words = 10

vocab = vectorizer.get_feature_names()
topic_words = {}
for topic, comp in enumerate(lda_model.components_): 
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    topic_words[topic] = [vocab[i] for i in word_idx]
    
for topic, words in topic_words.items():
    print('\nTopic: %d' % topic)
    print('%s' % ', '.join(words))


Topic: 0
power, law, act, state, court, office, constitution, case, may, officer

Topic: 1
court, judgment, case, justice, circuit, defendant, note, error, cause, plaintiff

Topic: 2
court, judgment, defendant, plaintiff, say, case, make, law, plea, act

Topic: 3
court, defendant, plaintiff, action, case, note, evidence, judgment, give, may


### Consider only words of interest
We now print the word distribution, considering only interesting words

In [21]:
narcotics = ['cannabis', 'cocaine', 'methamphetamine', 'drugs', 'drug', 'marijuana', 
             'ecstasy', 'lsd', 'ketamine', 'heroin', 'fentanyl', 'overdose']

weapons = ['gun', 'knife', 'weapon', 'firearm', 'rifle', 'carabine', 'shotgun', 'handgun', 
           'revolver', 'musket', 'pistol', 'derringer', 'assault', 'rifle', 'sword', 'blunt']

investigation = ['gang', 'mafia', 'serial',  'killer', 'rape', 'theft', 'recidivism', 
                 'arrest', 'robbery', 'cybercrime', 'cyber', 'crime']

interesting_set = set(narcotics + weapons + investigation)

In [23]:
topic_words = {}
for topic, comp in enumerate(lda_model.components_): 
    word_idx = np.argsort(comp)[::-1]
    topic_words[topic] = [w for w in [vocab[i] for i in word_idx] if w in interesting_set][:n_top_words]
    
for topic, words in topic_words.items():
    print('\nTopic: %d' % topic)
    print('%s' % ', '.join(words))


Topic: 0
crime, arrest, theft, assault, robbery

Topic: 1
arrest, crime, assault, robbery, theft

Topic: 2
arrest, assault, crime, robbery, theft

Topic: 3
assault, crime, arrest, robbery, theft


We can see that the topics blends together even considering only the words of interest, LDA must be guided. 

## Guided LDA approach
We now guide the lda process by setting some seeds, exploiting the model defined by the GuidedLDA package.

In [24]:
word2id = dict((v, idx) for idx, v in enumerate(vocab))

In [25]:
seed_topic_list = [narcotics, investigation, weapons]
seed_topics = {}

for i, st in enumerate(seed_topic_list):
    for word in st:
        if word in word2id:
            seed_topics[word2id[word]] = i
        else:
            print(f"{word} not found in vocabulary")

cannabis not found in vocabulary
cocaine not found in vocabulary
methamphetamine not found in vocabulary
drugs not found in vocabulary
drug not found in vocabulary
marijuana not found in vocabulary
ecstasy not found in vocabulary
lsd not found in vocabulary
ketamine not found in vocabulary
heroin not found in vocabulary
fentanyl not found in vocabulary
overdose not found in vocabulary
gang not found in vocabulary
mafia not found in vocabulary
serial not found in vocabulary
killer not found in vocabulary
rape not found in vocabulary
recidivism not found in vocabulary
cybercrime not found in vocabulary
cyber not found in vocabulary
gun not found in vocabulary
knife not found in vocabulary
weapon not found in vocabulary
firearm not found in vocabulary
rifle not found in vocabulary
carabine not found in vocabulary
shotgun not found in vocabulary
handgun not found in vocabulary
revolver not found in vocabulary
musket not found in vocabulary
pistol not found in vocabulary
derringer not found

In [26]:
glda_model = glda.GuidedLDA(n_topics=3, 
                       n_iter=5, 
                       random_state=0, 
                       refresh=10, 
                       alpha=alpha, 
                       eta=beta)

glda_model.fit(vectors, 
          seed_topics=seed_topics, 
          seed_confidence=0.99)

INFO:lda:n_documents: 440
INFO:lda:vocab_size: 7462
INFO:lda:n_words: 171104
INFO:lda:n_topics: 3
INFO:lda:n_iter: 5
INFO:lda:<0> log likelihood: -1448480
INFO:lda:<4> log likelihood: -1376458


In [29]:
topic_word = glda_model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][::-1]
    topic_words = [w for w in topic_words if w in interesting_set][:n_top_words]
    print(f"Topic {i}:\n{' '.join(topic_words)}")

Topic 0:
robbery arrest assault crime theft
Topic 1:
arrest crime assault robbery theft
Topic 2:
assault theft crime arrest robbery


## LSI

In [ ]:
reverse_vocabulary = { dv.vectorizer.vocabulary_[k]:k for k in dv.vectorizer.vocabulary_}

In [ ]:
model = LsiModel(vectors.transpose(), id2word=reverse_vocabulary, num_topics=numTopics) 
topics = model.get_topics()

In [ ]:
topWords = []
for topicno in range(numTopics):
    print('Topic {}'.format(topicno))
    print([(x, round(y, 2)) for x, y in model.show_topic(topicno, topn=10)], '\n')
    topWords.append([(x) for x, y in model.show_topic(topicno, topn=10)])